In [1]:
import os
import glob
import numpy as np

import re
from collections import defaultdict

In [2]:
main_path = 'experiments_logs'
loss_mode = 'MSE'
datasets = 'Epilepsy_2_Epilepsy'

# datasets = 'SleepEEG_2_SleepEEG'
n_epoch=2

In [3]:
loss_lst = ['MSE', 'SDSC','Hybrid']
all_resutls = defaultdict(dict)

for loss_mode in loss_lst:
    path = glob.glob(os.path.join(main_path, loss_mode, datasets, 'run1',"*"))[0]
    print(path)
    for p in os.listdir(path):
        if p.endswith("log"):
            log_path = os.path.join(path, p)
            break
            
    with open(log_path, "r") as f:
        logs = f.readlines()
    
    
    results = defaultdict(dict)
    current_epoch = None
    
    for line in logs:
        # Pre-training Epoch ?? ??
        pretrain_match = re.search(r"Pre-training Epoch: (\d+)", line)
        if pretrain_match:
            current_epoch = int(pretrain_match.group(1))
            mae_match = re.search(r"MAE Loss\s*:\s*([\d.]+)", line)
            rb_match = re.search(r"RB Loss\s*:\s*([\d.]+)", line)
            sd_metric_match = re.search(r"SD Metric:\s*([\d.]+)", line)
            dtw_metric_match = re.search(r"DTW Loss:\s*([\d.]+)", line)
            cl_metric_match = re.search(r"CL Loss:\s*([\d.]+)", line)
            if mae_match:
                results[current_epoch]["MAE"] = float(mae_match.group(1))
            if rb_match:
                results[current_epoch]["MSE"] = float(rb_match.group(1))
            if sd_metric_match:
                results[current_epoch]["SD"] = float(sd_metric_match.group(1))
    
            if dtw_metric_match:
                results[current_epoch]["DTW"] = float(dtw_metric_match.group(1))
            if cl_metric_match:
                results[current_epoch]["CL"] = float(cl_metric_match.group(1))
    
        # Fine-tuning ?? ?? (??: EP5 - Best Testing: Acc=... | Precision=... | ...)
        eval_match = re.search(
            r"EP\s*(\d+)\s*-\s*Best Testing:\s*Acc\s*=\s*([\d.]+)\s*\|\s*Precision\s*=\s*([\d.]+)\s*\|\s*Recall\s*=\s*([\d.]+)\s*\|\s*F1\s*=\s*([\d.]+)",
            line
        )
        if eval_match:
            epoch = int(eval_match.group(1))
            results[epoch]["Acc"] = float(eval_match.group(2))
            results[epoch]["Precision"] = float(eval_match.group(3))
            results[epoch]["Recall"] = float(eval_match.group(4))
            results[epoch]["F1"] = float(eval_match.group(5))
    
    # ?? ??
    sorted_results = sorted(results.items())
    
    # ??
    pre_results = defaultdict(list)

    print(f"LOSS MODE : {loss_mode}")
    print(f"{'Epoch':>5} | {'CL':>8} |{'MAE':>8} | {'MSE':>8}  | {'SD':>8} | {'DTW':>12} | {'Acc':>8} | {'Precision':>10} | {'Recall':>8} | {'F1':>8}")
    print("-" * 120)
    for epoch, metrics in sorted_results:
        if "MAE" in metrics:
            mae = f"{metrics['MAE']:.4f}"  
            # mae_lst.append(float(mae))
            pre_results['MAE'].append(float(mae))
        else:
            mae = "-"
        if "MSE" in metrics:
            mse = f"{metrics['MSE']:.4f}"  
            # mae_lst.append(float(mae))
            pre_results['MSE'].append(float(mse))
        else:
            mse = "-"
        if "SD" in metrics:
            sd = f"{metrics['SD']:.4f}" 
            pre_results['SD'].append(float(sd))
        else:
            sd =  "-"
        if "DTW" in metrics:
            dtw = f"{metrics['DTW']:.4f}"
            pre_results['DTW'].append(float(dtw))
        else:
            dtw = "-"
        if "CL" in metrics:
            cl = f"{metrics['CL']:.4f}"
            pre_results['CL'].append(float(cl))
        else:
            cl = "-"
        if "Acc" in metrics:
            acc = f"{metrics['Acc']:.4f}"  
            pre_results['Acc'].append(float(acc))
        else:
            acc = "-"
        if "Precision" in metrics:
            prec = f"{metrics['Precision']:.4f}" 
            pre_results['Precision'].append(float(prec))
        else:
            prec = "-"
        if "Recall" in metrics:
            recall = f"{metrics['Recall']:.4f}"  
            pre_results['Recall'].append(float(recall))
        else:
            recall = "-"
        if "F1" in metrics:
            f1 = f"{metrics['F1']:.4f}"  
            pre_results['F1'].append(float(f1))
        else:
            f1 = "-"
        if epoch % n_epoch == 0:
            print(f"{epoch:5} | {cl:>8} | {mae:>8} | {mse:>8} | {sd:>8} | {dtw:>12} | {acc:>8} | {prec:>10} | {recall:>8} | {f1:>8}")
    print('\n')
    all_resutls[loss_mode] = pre_results

experiments_logs/MSE/Epilepsy_2_Epilepsy/run1/pre_train_None_pt_0.5_0.0001_20_ft_0.0001_100
LOSS MODE : MSE
Epoch |       CL |     MAE |      MSE  |       SD |          DTW |      Acc |  Precision |   Recall |       F1
------------------------------------------------------------------------------------------------------------------------
    2 |  14.9596 |   0.4104 |   0.2614 |   0.2871 |    5954.7295 |  93.1961 |    94.4035 |  83.8597 |  87.9108
    4 |  14.8798 |   0.3457 |   0.1840 |   0.3989 |    4191.3838 |  93.6690 |    94.4482 |  85.2543 |  88.9277
    6 |  14.6270 |   0.3075 |   0.1402 |   0.4861 |    3193.5630 |  94.2032 |    94.6071 |  86.7705 |  90.0230
    8 |  14.8058 |   0.2790 |   0.1098 |   0.5437 |    2501.3350 |  94.2032 |    94.6071 |  86.7705 |  90.0230
   10 |  14.6419 |   0.2637 |   0.0971 |   0.5811 |    2213.1938 |  94.2032 |    94.6071 |  86.7705 |  90.0230
   12 |  14.7071 |   0.2569 |   0.0910 |   0.6014 |    2072.8599 |  94.2032 |    94.6071 |  86.7705 |  90